In [ ]:
!pip install yolov5
!pip uninstall yolov5
!pip install yolov5
import os
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
class WebElementsDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform

        self.image_paths = []
        self.labels = []

        for root, dirs, files in os.walk(data_dir):
            for file in files:
                if file.endswith('.jpg') or file.endswith('.png'):
                    image_path = os.path.join(root, file)
                    self.image_paths.append(image_path)

                    label = os.path.basename(root)
                    self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        image = Image.open(image_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        label = self.labels[index]

        return image, label

transform = transforms.Compose([
    transforms.Resize((416, 416)),
    transforms.ToTensor(),
])

# Define output directory for synthetic dataset
output_dir = 'synthetic_dataset'

# Function to generate synthetic images
def generate_synthetic_images(output_dir, num_images_per_class=100):
    web_elements = {
        'Breadcrumb': 'blue',
        'Checkbox': 'green',
        'Container': 'yellow',
        'Dropdown': 'orange',
        'Button': 'red',
        'Logo': 'cyan',
        'Navigation Dots': 'magenta',
        'Text': 'purple',
        'Link': 'brown',
        'Input Field': 'pink',
        'Radio': 'gray',
        'Modal': 'lime'
    }

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for element, color in web_elements.items():
        element_dir = os.path.join(output_dir, element)
        os.makedirs(element_dir, exist_ok=True)

        for i in range(num_images_per_class):
            width = np.random.randint(50, 300)
            height = np.random.randint(50, 300)
            image = Image.new('RGB', (width, height), color='white')
            draw = ImageDraw.Draw(image)

            xmin = np.random.randint(5, width - 25)
            ymin = np.random.randint(5, height - 25)
            xmax = np.random.randint(xmin + 20, width - 5)
            ymax = np.random.randint(ymin + 20, height - 5)
            bbox = (xmin, ymin, xmax, ymax)

            draw.rectangle(bbox, outline=color, width=2)

            font = ImageFont.load_default()
            label = element
            draw.text((xmin + 5, ymin + 5), label, fill=color, font=font)

            image_path = os.path.join(element_dir, f'image_{i}.jpg')
            image.save(image_path)

# Generate synthetic images for the sample dataset
generate_synthetic_images(output_dir, num_images_per_class=100)

# Load synthetic dataset
train_dataset = WebElementsDataset(data_dir=output_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# Define YOLOv5 model
model = YOLOv5(num_classes=13)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Save trained model weights
torch.save(model.state_dict(), 'yolov5_web_elements.pth')

def detect_objects_and_save(image_path, model, output_path, confidence_threshold=0.5):
    image = Image.open(image_path)

    preprocess = transforms.Compose([
        transforms.Resize((416, 416)),
        transforms.ToTensor(),
    ])
    image_tensor = preprocess(image).unsqueeze(0)

    model.eval()
    with torch.no_grad():
        outputs = model(image_tensor)

    bounding_boxes = [(50, 50, 100, 100), (200, 200, 300, 300)]  # Example bounding boxes
    class_predictions = [0, 1]  # Example class predictions

    annotated_image = image.copy()
    draw = ImageDraw.Draw(annotated_image)
    for box, class_pred in zip(bounding_boxes, class_predictions):
        if class_pred == 0:  # Example condition for filtering based on class
            draw.rectangle(box, outline="red")
            draw.text((box[0], box[1]), "Object", fill="red")

    annotated_image.save(output_path)
    print


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.4/953.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 723.1/723.1 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 14